In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 예외처리
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException, TimeoutException, NoSuchWindowException, WebDriverException

import time
import pandas as pd

In [ ]:
# 각자 맞게 파일 path 설정
file = r"C:\Users\dldpw\Desktop\STUDY\Study\빅콘테스트\data\df2.csv"

# csv 파일 읽기
jejuFood_raw = pd.read_csv(file, encoding='utf-8')
jejuFood = jejuFood_raw.copy()
print(jejuFood.shape)
jejuFood.head(5)

In [ ]:
jejuFood[2000:2001]

In [ ]:
# 지도에서 원하는 관광지 선택하기 함수
def click_spot(dong):
    # 원하는 관광지명의 시군구명이 같은 항목 선택하기
    addrs = driver.find_elements(By.CLASS_NAME, 'item_address._btnAddress')

    # 반복문 돌려서 원하는 구 이름 찾으면 선택하기
    for i, addr in enumerate(addrs):
        try:
            # 제주 음식점이면
            if ('제주시' in addr.text) or ('서귀포시' in addr.text):
                button = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, f'#ct > div.search_listview._content._ctList > ul > li:nth-child({i+1}) > div.item_info > div.item_info_inn > div > a')))
                button.click()

                # 지번으로 읍면동 맞는지 확인
                지번 = driver.find_element(By.CSS_SELECTOR, f'#ct > div.search_listview._content._ctList > ul > li:nth-child({i+1}) > div.item_info > div.wrap_bx_address._addressBox > div > p:nth-child(2)').text
                if dong in 지번:
                    # 찾고 있는 관광지 클릭 준비
                    spot_button = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, f'#ct > div.search_listview._content._ctList > ul > li:nth-child({i+1}) > div.item_info > a.a_item.a_item_distance._linkSiteview')))
                    break
                else:
                    continue
            else:
                continue
        except:
            print('해당 관광지 없음.')

    return spot_button

In [ ]:
# 카테고리 추출 후, 리뷰 탭 선택하기 함수
def click_reviewTab():
    tabs = driver.find_elements(By.CSS_SELECTOR, 'a.tpj9w._tab-menu > span.veBoZ')
    for i, tab in enumerate(tabs):
        try:
            if '리뷰' in tab.text:
                button = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.CSS_SELECTOR, f'div.place_fixed_maintab > div > div > div > div > a:nth-child({i+1})')))
                break
            else:
                continue
        except:
            print('리뷰 탭 없음.')

    return button

In [ ]:
# 리뷰 크롤링
def get_reviews(spot, addr, naver_name, category, review_k, review_list):
    reviews = driver.find_elements(By.CSS_SELECTOR, 'div.place_section.k1QQ5 > div > ul > li')

    if len(reviews) <= 100:
        reviews_num = len(reviews)
    else:
        reviews_num = 100

    contexts = ""

    for i in range(0, reviews_num):
        try:
            # 예약유무 항목 확인
            reservation = driver.find_element(By.CSS_SELECTOR, f'div.place_section.k1QQ5 > div > ul > li:nth-child({i+1}) > div.pui__-0Ter1 > a > span:nth-child(1) > em').text
            contexts += (reservation + " ")
        except:
            # 예약유무 없으면 패스
            print('예약유무 항목이 없습니다.')

        try:
            # 대기시간 항목 확인
            waiting = driver.find_element(By.CSS_SELECTOR, f'div.place_section.k1QQ5 > div > ul > li:nth-child({i+1}) > div.pui__-0Ter1 > a > span:nth-child(2) > em').text
            contexts += (waiting + " ")

        except:
            # 대기시간 없으면 패스
            print('대기시간 항목이 없습니다.')

        try:
            # 목적 항목 확인
            purpose = driver.find_element(By.CSS_SELECTOR, f'div.place_section.k1QQ5 > div > ul > li:nth-child({i+1}) > div.pui__-0Ter1 > a > span:nth-child(3) > em').text
            contexts += (purpose + " ")
        except:
            # 목적 없으면 패스
            print('목적 항목이 없습니다.')

        try:
            # 구성원 항목 확인
            member = driver.find_element(By.CSS_SELECTOR, f'div.place_section.k1QQ5 > div > ul > li:nth-child({i+1}) > div.pui__-0Ter1 > a > span:nth-child(4) > em').text
            contexts += (member + " ")
        except:
            # 구성원 없으면 패스
            print('구성원 항목이 없습니다.')

        try:
            # 리뷰 항목 확인
            context = driver.find_element(By.CSS_SELECTOR, f'div.place_section.k1QQ5 > div > ul > li:nth-child({i+1}) > div.pui__vn15t2 > a.pui__xtsQN-').text
            contexts += (context + " / ")
        except:
            # 리뷰 없으면 패스
            print('리뷰 항목이 없습니다.')

    # 네이버 리뷰 내용 추가
    review_list.append({'name':spot, 'address':addr, 'NAVER_name':naver_name, 'category':category, 'review_num':review_k, 'reviews':contexts})

    return review_list



In [ ]:
# 에러 안 난 결과 리스트
results = []
# 에러 난 결과 리스트
errors = []

# 사용할 DF
df = jejuFood

# 웹 드라이버 실행
driver = webdriver.Chrome()
# 웹 드라이버 최대화
driver.maximize_window()

# 반복문으로 음식점 크롤링 진행하기
for i in range(2000, 2501):
    # 관광지명 가져오기
    spot = df.iloc[i, 0]
    # 주소 가져오기
    addr = df.iloc[i, 2]
    # 동 이름 가져오기
    dong = addr.split()[2]

    try:
        # spot 찾기
        driver.get(f"https://m.map.naver.com/search2/search.naver?query={spot}")
        time.sleep(2)
        print('-'*20, i+1, ':', spot, f'({dong})', '-'*20)

        # 해당 음식점 클릭
        button = click_spot(dong)
        if button != None:
            button.click()
            time.sleep(3)

            # 네이버 음식점명 가져오기
            naver_name = WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.CLASS_NAME, 'GHAhO'))).text
            print(naver_name)

            # 카테고리 가져오기
            category = WebDriverWait(driver, 3).until(EC.visibility_of_element_located((By.CLASS_NAME, 'lnJFt'))).text
            print(category)

            # 리뷰 탭 클릭
            button = click_reviewTab()
            if button != None:
                button.click()
                time.sleep(3)

                # 리뷰개수 가져오기
                review_k = driver.find_element(By.CLASS_NAME, 'place_section_count')
                if review_k:
                    print(review_k.text)
                    review_k = review_k.text
                else:
                    print('해당 클래스 없음')

                # 리뷰항목 더보기 누르기
                total_num = 0

                while True:
                    if total_num < 100:
                        reviews = driver.find_elements(By.CSS_SELECTOR, 'div.place_section.k1QQ5 > div > ul > li')
                        total_num = len(reviews)

                        try:
                            # 더보기 버튼 찾기
                            button = WebDriverWait(driver, 3).until(EC.element_to_be_clickable((By.CLASS_NAME, 'fvwqf')))
                            driver.execute_script("arguments[0].scrollIntoView({block: 'center'});", button)
                            time.sleep(3)
                            button.click()
                            time.sleep(2)

                        except:
                            print('더보기 버튼이 더이상 없습니다.')
                            break
                    else:
                        break

                results = get_reviews(spot, addr, naver_name, category, review_k, results)
                print(results)

            else:
                print('"리뷰 탭"이 없어서 다음으로 넘어갑니다.')

        else:
            print('"해당 음식점"이 없어서 다음으로 넘어갑니다.')

    except:
        print('해당 spot을 찾을 수 없습니다.')
        errors.append(spot)

driver.quit()




In [ ]:
# 크롤링된 음식점 df
results_df = pd.DataFrame(results)
results_df.to_csv(r"C:\Users\dldpw\Desktop\STUDY\Study\빅콘테스트\data\df_6.csv", encoding='utf-8', index=0)

In [ ]:
# 크롤링된 음식점 df
results_df = pd.DataFrame(results)
results_df.to_csv(r"C:\Users\dldpw\Desktop\STUDY\Study\빅콘테스트\data\df_6_1.csv", encoding='utf-8', index=0)